## SGRank

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import itertools
import logging
import math
import operator
from decimal import Decimal

import networkx as nx
import numpy as np
from cytoolz import itertoolz

import compat
import extract
import network
#import similarity
import vsm

LOGGER = logging.getLogger(__name__)

In [2]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [5]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

import nltk
from keras.preprocessing.text import Tokenizer

C:\Users\hp\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.
C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Text Used

Machine learning is a field of computer science that uses statistical techniques to give computer systems the ability to "learn" (e.g., progressively improve performance on a specific task) with data, without being explicitly programmed.The name machine learning was coined in 1959 by Arthur Samuel.Evolved from the study of pattern recognition and computational learning theory in artificial intelligence, machine learning explores the study and construction of algorithms that can learn from and make predictions on data – such algorithms overcome following strictly static program instructions by making data-driven predictions or decisions, through building a model from sample inputs. Machine learning is employed in a range of computing tasks where designing and programming explicit algorithms with good performance is difficult or infeasible; example applications include email filtering, detection of network intruders, and computer vision.Machine learning is closely related to (and often overlaps with) computational statistics, which also focuses on prediction-making through the use of computers. It has strong ties to mathematical optimization, which delivers methods, theory and application domains to the field. Machine learning is sometimes conflated with data mining, where the latter subfield focuses more on exploratory data analysis and is known as unsupervised learning.Within the field of data analytics, machine learning is a method used to devise complex models and algorithms that lend themselves to prediction; in commercial use, this is known as predictive analytics. These analytical models allow researchers, data scientists, engineers, and analysts to "produce reliable, repeatable decisions and results" and uncover "hidden insights" through learning from historical relationships and trends in the data.

In [3]:
text = "Machine learning is a field of computer science that uses statistical techniques to give computer systems the ability to learn (e.g., progressively improve performance on a specific task) with data, without being explicitly programmed.The name machine learning was coined in 1959 by Arthur Samuel.Evolved from the study of pattern recognition and computational learning theory in artificial intelligence, machine learning explores the study and construction of algorithms that can learn from and make predictions on data – such algorithms overcome following strictly static program instructions by making data-driven predictions or decisions, through building a model from sample inputs. Machine learning is employed in a range of computing tasks where designing and programming explicit algorithms with good performance is difficult or infeasible; example applications include email filtering, detection of network intruders, and computer vision.Machine learning is closely related to (and often overlaps with) computational statistics, which also focuses on prediction-making through the use of computers. It has strong ties to mathematical optimization, which delivers methods, theory and application domains to the field. Machine learning is sometimes conflated with data mining, where the latter subfield focuses more on exploratory data analysis and is known as unsupervised learning.Within the field of data analytics, machine learning is a method used to devise complex models and algorithms that lend themselves to prediction; in commercial use, this is known as predictive analytics. These analytical models allow researchers, data scientists, engineers, and analysts to produce reliable, repeatable decisions and results and uncover hidden insights through learning from historical relationships and trends in the data."

In [4]:
doc = nlp(u"Machine learning is a field of computer science that uses statistical techniques to give computer systems the ability to learn (e.g., progressively improve performance on a specific task) with data, without being explicitly programmed.The name machine learning was coined in 1959 by Arthur Samuel.Evolved from the study of pattern recognition and computational learning theory in artificial intelligence, machine learning explores the study and construction of algorithms that can learn from and make predictions on data – such algorithms overcome following strictly static program instructions by making data-driven predictions or decisions, through building a model from sample inputs. Machine learning is employed in a range of computing tasks where designing and programming explicit algorithms with good performance is difficult or infeasible; example applications include email filtering, detection of network intruders, and computer vision.Machine learning is closely related to (and often overlaps with) computational statistics, which also focuses on prediction-making through the use of computers. It has strong ties to mathematical optimization, which delivers methods, theory and application domains to the field. Machine learning is sometimes conflated with data mining, where the latter subfield focuses more on exploratory data analysis and is known as unsupervised learning.Within the field of data analytics, machine learning is a method used to devise complex models and algorithms that lend themselves to prediction; in commercial use, this is known as predictive analytics. These analytical models allow researchers, data scientists, engineers, and analysts to produce reliable, repeatable decisions and results and uncover hidden insights through learning from historical relationships and trends in the data.")

## terms_to_semantic_network

### Helper Functions 

In [6]:
def get_word_embedding(terms):
    filename = 'C:/Users/hp/Word_embeddings/GoogleNews-vectors-negative300.bin'
    model = gensim.models.KeyedVectors.load_word2vec_format( filename , binary=True)

    words_to_index = {}
    i = 0;
    for word in terms:
        if not word in words_to_index:
            words_to_index[str(word)] = i
            i = i + 1
        else:
            continue
   
   
    embedding_matrix = np.zeros(( len(words_to_index) , 300 ))
    for word , i in words_to_index.items():
        try:
            embedding_vector = model[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        

    return (embedding_matrix , words_to_index)
    

    

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0 :
        return 0
    
    return dot_product / (norm_a * norm_b)


def get_cosine_mat(windows , embedding_matrix , words_to_index ):
    from scipy import spatial
    cosine_mat = collections.defaultdict(lambda: collections.defaultdict(float))
  
    for window in windows:
        for w1, w2 in itertools.combinations(sorted(window), 2):
               cosine_mat[w1][w2] = cos_sim(embedding_matrix[words_to_index[w1]] , embedding_matrix[words_to_index[w2]] )
           
            
    return cosine_mat        

In [8]:
from collections import defaultdict
import math

def count_of_single( Tokens , freqThreshold):          
    word_freq = defaultdict(int)
    fdist = defaultdict(int)
    for a in Tokens:
        fdist[a] += 1 
    
    for word , freq in sorted(fdist.items(), key=lambda k__v: (k__v[1],k__v[0])):
        if freq > freqThreshold:
            word_freq[word] = freq
    return word_freq

def count_of_bigrams( Tokens ,freqThreshold):
        bigram_freq = defaultdict(int)

        b = nltk.collocations.BigramCollocationFinder.from_words(Tokens)
        b.apply_freq_filter(freqThreshold)
       
        for bigram, freq in b.ngram_fd.items():
                bigram=" ".join([bigram[0], bigram[1]])
                bigram_freq[bigram] = freq
        return bigram_freq
    
def pmi(w1, w2, unigram_freq , bigram_freq):

    prob_word1 = unigram_freq[w1] / float(sum(unigram_freq.values()))
    prob_word2 = unigram_freq[w2] / float(sum(unigram_freq.values()))
    prob_word1_word2 = bigram_freq[" ".join([w1, w2])] / float(sum(bigram_freq.values()))

   
    #print("PMI FOR W1 AND W2 ")
    ##print(w1 , w2)
    #print("Probability of w1 ")
    #print( prob_word1)
    #print("Probability of w2 ")
    #print( prob_word2)
    #print("Probability of w1 and w2 joint ")
    #print( prob_word1_word2 )
    if prob_word1_word2 == 0 :
        return 0
    #print(math.log(prob_word1_word2/float(prob_word1*prob_word2),2))
    try:

        return math.log(prob_word1_word2/float(prob_word1*prob_word2),2)

    except: # Occurs when calculating PMI for Out-of-Vocab words.

        return 0

def get_pmi_mat(terms , windows ):
    pmi_mat = collections.defaultdict(lambda: collections.defaultdict(float))
    unigram_freq = count_of_single( terms , 0)
    bigram_freq = count_of_bigrams( terms , 0)
   
    for window in windows:
        #print(" IN GET_PMI WINDOW IN WINDOWS")
        for w1, w2 in itertools.combinations(sorted(window), 2):
            pmi_mat[w1][w2] = pmi( w1 , w2 ,  unigram_freq , bigram_freq )
            
            
    return pmi_mat        
    

In [9]:
def get_final_weights( cosine_mat , pmi_mat ):
    
    final_weights = collections.defaultdict(lambda: collections.defaultdict(float))
    for (i , j) , (k , l) in zip(cosine_mat.items() ,pmi_mat.items()):
        for (a , b) , (c , d) in zip(j.items() , l.items()):
                final_weights[i][a] = b*d 
              
          
    return final_weights
       

In [10]:
def get_word_embedding_phrases(terms):
    filename = 'C:/Users/hp/Word_embeddings/GoogleNews-vectors-negative300.bin'
    model = gensim.models.KeyedVectors.load_word2vec_format( filename , binary=True)

   
    words_to_index_1 = {}
    i = 0;
    for a in terms:
        for b in a.split():
            if not b in words_to_index_1:
                words_to_index_1[str(b)] = i
                i = i + 1
            else:
                continue   
    """   
    print("words_to_index in phrase embeddings")
    for k, v in words_to_index_1.items():
        print(k, v)
        
    print(" LENGTH OF WORS_TO_INDEX IS : ")
    print(len(words_to_index_1))
    """
    
    embedding_matrix = np.zeros(( len(words_to_index_1) , 300 ))
    for word , i in words_to_index_1.items():
        try:
            embedding_vector = model[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
       
    
    phrases_to_index = {}
    i = 0;
    for a in terms:
            if not a in phrases_to_index:
                phrases_to_index[str(a)] = i
                i = i + 1
            else:
                continue 
    """    
    print()    
    print("phrases_to_index in phrase embeddings")
    print()
    for k, v in phrases_to_index.items():
        print(k, v) 
        
    print(" LENGTH OF PHRASES_TO_INDEX IS : ")
    print(len( phrases_to_index ))    
    """
    
    embedding_matrix_phrases = np.zeros(( len(words_to_index_1) , 300 ))
    for phrase , i in phrases_to_index.items():
        embedding_vector_final = np.zeros(300)
        for a in phrase.split():
            try:
                embedding_vector = model[a] 
            except KeyError:
                embedding_vector = None    
            if embedding_vector is not None:
                embedding_vector_final = np.add( embedding_vector_final , embedding_vector )   
        embedding_matrix_phrases[i] = embedding_vector_final
        
           
    return  embedding_matrix ,  words_to_index_1 , embedding_matrix_phrases ,  phrases_to_index
        

In [11]:
def terms_to_semantic_network(terms, normalize='lemma', window_width=10, edge_weighting='cooc_freq' ,  phrases = False ):
   
    """
    Transform an ordered list of non-overlapping terms into a semantic network,
    where each term is represented by a node with weighted edges linking it to
    other terms that co-occur within ``window_width`` terms of itself.
    Args:
        terms (List[str] or List[``spacy.Token``])
        normalize (str or Callable): If 'lemma', lemmatize terms; if 'lower',
            lowercase terms; if false-y, use the form of terms as they appear
            in ``terms``; if a callable, must accept a ``spacy.Token`` and return
            a str, e.g. :func:`textacy.spacier.utils.get_normalized_text()`.
            .. note:: This is applied to the elements of ``terms`` *only* if
               it's a list of ``spacy.Token``.
        window_width (int): Size of sliding window over ``terms`` that determines
            which are said to co-occur. If 2, only immediately adjacent terms
            have edges in the returned network.
        edge_weighting ({'cooc_freq', 'binary'}): If 'cooc_freq', the nodes for
            all co-occurring terms are connected by edges with weight equal to
            the number of times they co-occurred within a sliding window;
            if 'binary', all such edges have weight = 1.
    Returns:
        ``networkx.Graph``: Nodes in this network correspond to individual terms;
        those that co-occur are connected by edges with weights determined
        by ``edge_weighting``.
    Notes:
        - Be sure to filter out stopwords, punctuation, certain parts of speech, etc.
          from the terms list before passing it to this function
        - Multi-word terms, such as named entities and compound nouns, must be merged
          into single strings or spacy.Tokens beforehand
        - If terms are already strings, be sure to have normalized them so that
          like terms are counted together; for example, by applying
          :func:`textacy.spacier.utils.get_normalized_text()`
    """
    if window_width < 2:
        raise ValueError(
            '`window_width` = {} is invalid; value must be >= 2'.format(window_width))
    if not terms:
        raise ValueError(
            '`terms` = {} is invalid; it must contain at least 1 term '
            'in the form of a string or spacy token'.format(terms))

    # if len(terms) < window_width, cytoolz throws a StopIteration error
    # which we don't want
    if len(terms) < window_width:
        LOGGER.info(
            '`terms` has fewer items (%s) than the specified `window_width` (%s); '
            'setting window width to %s',
            len(terms), window_width, len(terms))
        window_width = len(terms)

    if isinstance(terms[0], compat.unicode_):
        windows = itertoolz.sliding_window(window_width, terms)
        to_return = list(windows)
    elif isinstance(terms[0], SpacyToken):
        if normalize == 'lemma':
            windows = ((tok.lemma_ for tok in window)
                       for window in itertoolz.sliding_window(window_width, terms))
        elif normalize == 'lower':
            windows = ((tok.lower_ for tok in window)
                       for window in itertoolz.sliding_window(window_width, terms))
        elif not normalize:
            windows = ((tok.text for tok in window)
                       for window in itertoolz.sliding_window(window_width, terms))
        else:
            windows = ((normalize(tok) for tok in window)
                       for window in itertoolz.sliding_window(window_width, terms))
    else:
        raise TypeError(
            'items in `terms` must be strings or spacy tokens, not {}'.format(type(terms[0])))

    
    graph = nx.Graph()

    
    embedding_matrix_phrases = np.zeros(( len(terms) , 300 ))
    
    if edge_weighting == 'cooc_freq':
        cooc_mat = collections.defaultdict(lambda: collections.defaultdict(int))
        for window in to_return:
            for w1, w2 in itertools.combinations(sorted(window), 2):
                cooc_mat[w1][w2] += 1

                               
        graph.add_edges_from(
            (w1, w2, {'weight': weight})
            for w1, w2s in cooc_mat.items()
            for w2, weight in w2s.items())
    elif edge_weighting == 'binary':
        graph.add_edges_from(
            w1_w2 for window in to_return
            for w1_w2 in itertools.combinations(window, 2))
    
    elif edge_weighting == 'embedding' and phrases == False:
        embedding_matrix , word_to_index = get_word_embedding(terms)
        cosine_mat = get_cosine_mat(to_return , embedding_matrix  , word_to_index )
        pmi_mat = get_pmi_mat( terms , to_return )
        final_weights = get_final_weights( cosine_mat , pmi_mat )
        
        for window in to_return:
            for w1, w2 in itertools.combinations(sorted(window), 2):
                graph.add_weighted_edges_from(
                    (w1, w2,  weight)
                    for w1, w2s in final_weights.items()
                    for w2, weight in w2s.items())
      
    elif edge_weighting == 'embedding' and phrases == True:
        embedding_matrix ,  words_to_index_1 , embedding_matrix_phrases ,  phrases_to_index =  get_word_embedding_phrases(terms)
        cosine_mat = get_cosine_mat( to_return , embedding_matrix_phrases  , phrases_to_index )
        pmi_mat = get_pmi_mat( terms , to_return )
        final_weights = get_final_weights( cosine_mat , pmi_mat )
   
        for window in to_return:
            for w1, w2 in itertools.combinations(sorted(window), 2):
                graph.add_weighted_edges_from(
                    (w1, w2,  weight)
                    for w1, w2s in final_weights.items()
                    for w2, weight in w2s.items())
                
    
    #for (u , v , d) in graph.edges(data='weight'):
        #if d < 0:
            #d = d*(-1)
           # graph[u][v]['weight'] = d
            #print("CHANGED")
            #print()
    
  
   
    return graph

### SGRank

In [29]:
def sgrank(doc, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width=1500,
           n_keyterms=10, idf=None , edge_weighting = None , join_key_words = False):
    """
    Extract key terms from a document using the [SGRank]_ algorithm.
    Args:
        doc (``textacy.Doc`` or ``spacy.Doc``)
        ngrams (int or Set[int]): n of which n-grams to include; ``(1, 2, 3, 4, 5, 6)``
                (default) includes all ngrams from 1 to 6; `2`
                if only bigrams are wanted
        normalize (str or callable): If 'lemma', lemmatize terms; if 'lower',
            lowercase terms; if None, use the form of terms as they appeared in
            ``doc``; if a callable, must accept a ``spacy.Span`` and return a str,
            e.g. :func:`textacy.spacier.utils.get_normalized_text()`
        window_width (int): Width of sliding window in which term
            co-occurrences are determined to occur. Note: Larger values may
            dramatically increase runtime, owing to the larger number of
            co-occurrence combinations that must be counted.
        n_keyterms (int or float): Number of top-ranked terms to return as
            keyterms. If int, represents the absolute number; if float, must be
            in the open interval (0.0, 1.0), and is converted to an integer by
            ``int(round(len(doc) * n_keyterms))``
        idf (dict): Mapping of ``normalize(term)`` to inverse document frequency
            for re-weighting of unigrams (n-grams with n > 1 have df assumed = 1).
            Results are typically better with idf information.
    Returns:
        List[Tuple[str, float]]: sorted list of top ``n_keyterms`` key terms and
        their corresponding SGRank scores
    Raises:
        ValueError: If ``n_keyterms`` is a float but not in (0.0, 1.0] or
            ``window_width`` < 2.
    References:
        .. [SGRank] Danesh, Sumner, and Martin. "SGRank: Combining Statistical and
           Graphical Methods to Improve the State of the Art in Unsupervised Keyphrase
           Extraction". Lexical and Computational Semantics (* SEM 2015) (2015): 117.
    """
    n_toks = len(doc)
    
    if isinstance(n_keyterms, float):
        if not 0.0 < n_keyterms <= 1.0:
            raise ValueError('`n_keyterms` must be an int, or a float between 0.0 and 1.0')
        n_keyterms = int(round(n_toks * n_keyterms))
    
    if window_width < 2:
        raise ValueError('`window_width` must be >= 2')
    
    window_width = min(n_toks, window_width)
    min_term_freq = min(n_toks // 1000, 4)
    
    if isinstance(ngrams, int):
        ngrams = (ngrams,)

    # build full list of candidate terms
    # if inverse doc freqs available, include nouns, adjectives, and verbs;
    # otherwise, just include nouns and adjectives
    # (without IDF downweighting, verbs dominate the results in a bad way)
    include_pos = {'NOUN', 'PROPN', 'ADJ', 'VERB'} if idf else {'NOUN', 'PROPN', 'ADJ'}
    terms = itertoolz.concat(
        extract.ngrams(doc, n, filter_stops=True, filter_punct=True, filter_nums=False,
                       include_pos=include_pos, min_freq=min_term_freq)
                       for n in ngrams)

    #print(" TERMS : N-GRAMS")
    #a = list(terms)
    #print(a)
    
    
    # get normalized term strings, as desired
    # paired with positional index in document and length in a 3-tuple
    if normalize == 'lemma':
        terms = [(term.lemma_, term.start, len(term)) for term in terms]
    elif normalize == 'lower':
        terms = [(term.lower_, term.start, len(term)) for term in terms]
    elif not normalize:
        terms = [(term.text, term.start, len(term)) for term in terms]
    else:
        terms = [(normalize(term), term.start, len(term)) for term in terms]

    # pre-filter terms to the top N ranked by TF or modified TF*IDF
    n_prefilter_kts = max(3 * n_keyterms, 100)
    term_text_counts = collections.Counter(term[0] for term in terms)
    
    
    if idf:
        mod_tfidfs = {
            term: count * idf.get(term, 1) if ' ' not in term else count
            for term, count in term_text_counts.items()}
        terms_set = {
            term for term, _
            in sorted(mod_tfidfs.items(), key=operator.itemgetter(1), reverse=True)[:n_prefilter_kts]}
    else:
        terms_set = {term for term, _ in term_text_counts.most_common(n_prefilter_kts)}
        terms = [term for term in terms if term[0] in terms_set]

    term_weights = {}
    seen_terms = set()
    n_toks_plus_1 = n_toks + 1
    for term in terms:
        term_text = term[0]
        # we only want the *first* occurrence of a unique term (by its text)
        if term_text in seen_terms:
            continue
        seen_terms.add(term_text)
        pos_first_occ_factor = math.log(n_toks_plus_1 / (term[1] + 1))
        # TODO: assess how best to scale term len
        term_len = math.sqrt(term[2])  # term[2]
        term_count = term_text_counts[term_text]
        subsum_count = sum(term_text_counts[t2] for t2 in terms_set
                            if t2 != term_text and term_text in t2)
        term_freq_factor = term_count - subsum_count
        if idf and term[2] == 1:
            term_freq_factor *= idf.get(term_text, 1)
        term_weights[term_text] = term_freq_factor * pos_first_occ_factor * term_len
    
    
    terms = [term for term in terms if term_weights[term[0]] > 0]

    
    if edge_weighting == "embedding":
        #print("WOKING FINE")  
        phrases = []
        words = []
        
        terms_words = [ term[0] for term in terms ]
        for a in terms_words:
            if len([b  for b in a.split()]) == 1:
                words.append(a)
            if len([b  for b in a.split()]) > 1:
                phrases.append(a)        

        ## If only words are needed
        if edge_weighting == 'embedding' and join_key_words == False:    
            graph = terms_to_semantic_network( words , window_width = window_width, edge_weighting = edge_weighting , phrases = False)
    
        ## If keyphrases are needed
        elif edge_weighting == 'embedding' and join_key_words == True:
            graph = terms_to_semantic_network( phrases , window_width = window_width, edge_weighting = edge_weighting , phrases = True)  
    
        ## Using PageRank for ranking keywords/keyphrases
        word_ranks = nx.pagerank_scipy(graph , max_iter = 100 ,  weight = 'weight')
        return [(word, score) for word, score in
                sorted(word_ranks.items(), key=operator.itemgetter(1), reverse=True)[:n_keyterms]]
        

    # compute term weights from statistical attributes:
    # not subsumed frequency, position of first occurrence, and num words
    
    else : 
        n_coocs = collections.defaultdict(lambda: collections.defaultdict(int))
        sum_logdists = collections.defaultdict(lambda: collections.defaultdict(float))

        # iterate over windows
        log_ = math.log  # localize this, for performance
        for start_ind in compat.range_(n_toks):
            end_ind = start_ind + window_width
            window_terms = (term for term in terms
                            if start_ind <= term[1] <= end_ind)
            # get all token combinations within window
            for t1, t2 in itertools.combinations(window_terms, 2):
                n_coocs[t1[0]][t2[0]] += 1
                sum_logdists[t1[0]][t2[0]] += log_(window_width / max(abs(t1[1] - t2[1]), 1))
            if end_ind > n_toks:
                break

            # compute edge weights between co-occurring terms (nodes)
        edge_weights = collections.defaultdict(lambda: collections.defaultdict(float))
        for t1, t2s in sum_logdists.items():
            for t2 in t2s:
                edge_weights[t1][t2] = ((1.0 + sum_logdists[t1][t2]) / n_coocs[t1][t2]) * term_weights[t1] * term_weights[t2]
            # normalize edge weights by sum of outgoing edge weights per term (node)
                norm_edge_weights = []
            for t1, t2s in edge_weights.items():
                sum_edge_weights = sum(t2s.values())
                norm_edge_weights.extend((t1, t2, {'weight': weight / sum_edge_weights})
                                         for t2, weight in t2s.items())

            # build the weighted directed graph from edges, rank nodes by pagerank
            graph = nx.DiGraph()
            graph.add_edges_from(norm_edge_weights)
            term_ranks = nx.pagerank_scipy(graph)

            return sorted(term_ranks.items(), key=operator.itemgetter(1, 0), reverse=True)[:n_keyterms]
    

### Using embeddings

In [17]:
keyphrases = sgrank(doc, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width = 2, n_keyterms = 20, idf = None , edge_weighting = "embedding" , join_key_words = True) 

In [18]:
keyphrases

[('machine learning', 0.12780771437464766),
 ('computer science', 0.05221932114882506),
 ('statistical technique', 0.05221932114882506),
 ('computer system', 0.05221932114882506),
 ('specific task', 0.05221932114882506),
 ('sample input', 0.05221932114882506),
 ('arthur samuel', 0.05221932114882506),
 ('pattern recognition', 0.05221932114882506),
 ('computational learning', 0.05221932114882506),
 ('learning theory', 0.05221932114882506),
 ('program instruction', 0.05221932114882506),
 ('explicit algorithm', 0.05221932114882506),
 ('good performance', 0.05221932114882506),
 ('email filtering', 0.05221932114882506),
 ('network intruder', 0.05221932114882506),
 ('computer vision', 0.04710912678014121),
 ('static program', 0.03647922142751169),
 ('artificial intelligence', 0.02646188209902444),
 ('programming explicit', 0.023238661062800217),
 ('example application', 0.00783289817232376)]

In [19]:
keywords = sgrank(doc, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width = 2, n_keyterms = 20, idf = None , edge_weighting = "embedding" , join_key_words = False) 

In [20]:
keywords

[('tie', 0.04348985531602103),
 ('model', 0.04181281865616307),
 ('strong', 0.040243266227312134),
 ('analytic', 0.034382534800116035),
 ('analytical', 0.03400618093137236),
 ('application', 0.03357994074430049),
 ('mining', 0.03338276537803992),
 ('relationship', 0.032288908811599),
 ('use', 0.03228890881159899),
 ('reliable', 0.032288908811598974),
 ('datum', 0.0317891929019165),
 ('algorithm', 0.02906417551804707),
 ('domain', 0.028654509511509125),
 ('repeatable', 0.028354816324690106),
 ('data', 0.027011533007003728),
 ('theory', 0.02634262575431781),
 ('prediction', 0.025372802071835565),
 ('subfield', 0.02262360971499223),
 ('trend', 0.02224273635553439),
 ('difficult', 0.02212389380530973)]

###  Without embeddings

In [28]:
keyterms_without_embedding = sgrank(doc, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width = 2, n_keyterms = 20, idf = None , join_key_words = True) 

In [27]:
keyterms_without_embedding

[('machine learning', 0.27232895809719715),
 ('datum', 0.19858037029257036),
 ('computer science', 0.18047109960756447),
 ('computer', 0.1776795203062141),
 ('field', 0.17094005169645402)]